Entrenamiento para usar pýthon 

In [1]:
def convertToBase13(num):
    if num == 0:
        return "0"

    # Digits for base 13
    base13_digits = "0123456789ABC"  
    digits = ""
    positive = abs(num)
    
    while positive > 0:
        # Append digits/letters
        digits += base13_digits[positive % 13]  
        positive = positive // 13

    # Reverse the string at the end
    reversed_digits = digits[::-1]  
    if num < 0:
        return "-" + reversed_digits
    else:
        return reversed_digits

In [5]:
num = int(input("Escriba un numero"))
convertToBase13(num)

'A'

In [2]:
def func(x):
    if x == 1:
        def rv():
            print("X is equal to 1")
    else:
        def rv():
            print("X is not 1")

    return rv

In [5]:
new_func = func(2)
new_func

<function __main__.func.<locals>.rv()>

In [6]:
import inspect
from queue import Queue

print(inspect.getsource(Queue))

class Queue:
    '''Create a queue object with a given maximum size.

    If maxsize is <= 0, the queue size is infinite.
    '''

    def __init__(self, maxsize=0):
        self.maxsize = maxsize
        self._init(maxsize)

        # mutex must be held whenever the queue is mutating.  All methods
        # that acquire mutex must release it before returning.  mutex
        # is shared between the three conditions, so acquiring and
        # releasing the conditions also acquires and releases mutex.
        self.mutex = threading.Lock()

        # Notify not_empty whenever an item is added to the queue; a
        # thread waiting to get is notified then.
        self.not_empty = threading.Condition(self.mutex)

        # Notify not_full whenever an item is removed from the queue;
        # a thread waiting to put is notified then.
        self.not_full = threading.Condition(self.mutex)

        # Notify all_tasks_done whenever the number of unfinished tasks
        # drops to zero; th

# Validador de Agendamiento de Reuniones (Calendar Booking System)

## 📝 Descripción:
Construirás un sistema de agendamiento de reuniones que:

    - Permita a usuarios crear reuniones con fecha, hora de inicio y duración.

    - Valide que no haya solapamientos entre reuniones existentes.

    - Permita consultar las reuniones existentes de un día.

    - Permita cancelar reuniones.

## 🎯 Requisitos funcionales:
    
    - Crear reunión (schedule_meeting)

    - Parámetros: start_time, duration, description

    - Valida que no se cruce con ninguna otra existente.

    - Listar reuniones (list_meetings(date))

    - Devuelve lista ordenada por hora.

    - Cancelar reunión (cancel_meeting(id))

## 💡 Consideraciones técnicas:

    - Usa datetime para manejar fechas y horas.

    - Las reuniones duran una cantidad de minutos (duration).

    - Valida que una reunión no se cruce en el tiempo con otra.

In [1]:
from datetime import datetime, timedelta
from typing import List
import uuid

class Meeting:
    def __init__(self, start_time: datetime, duration: int, description: str):
        self.id = str(uuid.uuid4())
        self.start_time = start_time
        self.end_time = start_time + timedelta(minutes=duration)
        self.duration = duration
        self.description = description

    def __str__(self):
        return f"{self.description} ({self.start_time.strftime('%H:%M')} - {self.end_time.strftime('%H:%M')})"

class MeetingScheduler:
    def __init__(self):
        self.meetings: List[Meeting] = []

    def schedule_meeting(self, start: str, duration: int, description: str):
        start_dt = datetime.strptime(start, "%Y-%m-%d %H:%M")
        new_meeting = Meeting(start_dt, duration, description)

        for meeting in self.meetings:
            if self._overlaps(meeting, new_meeting):
                raise ValueError("La nueva reunión se cruza con una existente.")

        self.meetings.append(new_meeting)
        print(f"Reunión programada: {new_meeting}")

    def _overlaps(self, m1: Meeting, m2: Meeting) -> bool:
        return m1.start_time < m2.end_time and m2.start_time < m1.end_time

    def list_meetings(self, date: str):
        date_dt = datetime.strptime(date, "%Y-%m-%d").date()
        filtered = [m for m in self.meetings if m.start_time.date() == date_dt]
        for m in sorted(filtered, key=lambda x: x.start_time):
            print(f"[{m.id[:6]}] {m}")

    def cancel_meeting(self, meeting_id: str):
        before = len(self.meetings)
        self.meetings = [m for m in self.meetings if m.id != meeting_id]
        after = len(self.meetings)
        if before == after:
            print("ID no encontrado.")
        else:
            print("Reunión cancelada.")

In [3]:
calendar = MeetingScheduler()

calendar.schedule_meeting("2025-07-17 09:00", 60, "Reunión con cliente")
calendar.schedule_meeting("2025-07-17 10:30", 30, "Planificación interna")

Reunión programada: Reunión con cliente (09:00 - 10:00)
Reunión programada: Planificación interna (10:30 - 11:00)


In [5]:
calendar.list_meetings("2025-07-17")
# → Reunión con cliente (9:00 - 10:00), Planificación interna (10:30 - 11:00)

[57eb5e] Reunión con cliente (09:00 - 10:00)
[90a3e3] Planificación interna (10:30 - 11:00)


In [6]:
calendar.schedule_meeting("2025-07-17 09:30", 30, "Solapada")  
# → Error: se cruza con otra

ValueError: La nueva reunión se cruza con una existente.

# Diseña un sistema de detección de tasa de peticiones (Rate Limiter)

## 💼 Descripción:
Vas a construir un Rate Limiter, es decir, un componente que impide que un usuario (o IP, o servicio) haga demasiadas peticiones en un corto periodo de tiempo.

    - Este patrón es usado en producción por:

    - APIs públicas (OpenAI, Twitter, GitHub)

    - Servicios que protegen de abuso (login, spam, etc.)

## 🎯 Requisitos:
Solo se deben permitir N peticiones por usuario por ventana de tiempo T (por ejemplo, 5 peticiones cada 10 segundos).

    - Rechaza peticiones que excedan ese límite.

    - Opcional: Implementa variantes como sliding window o leaky bucket si quieres ir más lejos.

In [1]:
from collections import deque
from typing import Dict

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.user_requests: Dict[str, deque] = {}

    def allow_request(self, user_id: str, current_time: int) -> bool:
        """
        Registra y decide si la petición puede hacerse en el timestamp actual.
        """
        if user_id not in self.user_requests:
            self.user_requests[user_id] = deque()

        requests = self.user_requests[user_id]

        # Elimina peticiones fuera de la ventana de tiempo
        while requests and current_time - requests[0] >= self.window_seconds:
            requests.popleft()

        if len(requests) < self.max_requests:
            requests.append(current_time)
            return True
        return False

In [4]:
limiter = RateLimiter(max_requests=3, window_seconds=10)

#print(limiter.allow_request("user1", 1))   # True
#print(limiter.allow_request("user1", 2))   # True
#print(limiter.allow_request("user1", 3))   # True
#print(limiter.allow_request("user1", 4))   # False (excede el límite)
print(limiter.allow_request("user1", 12))  # True (ventana reseteada)


True
